In [2]:
import argparse
import importlib
import Semantic_Meaningfulness_v2
from Semantic_Meaningfulness_v2 import Sematic
importlib.reload(Semantic_Meaningfulness_v2)
import carla.recourse_methods.catalog as recourse_catalog
from carla.data.causal_model import CausalModel
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
from carla.data.catalog import CsvCatalog
from carla.evaluation.catalog.success_rate import SuccessRate
#from carla.recourse_methods.catalog import recourse_catalog
#from carla.recourse_methods.catalog.roar.model import Roar
from carla.recourse_methods.catalog.causal_recourse import (
    CausalRecourse,
    constraints,
    samplers,
)
#from carla.data.causal_model.synthethic_data import SCMDataset
from carla.recourse_methods import GrowingSpheres
from causal_recourse_do_calculus.model import CausalRecourse_DoCalculus
from carla import Benchmark
import numpy as np 
import pandas as pd
import torch
import random
import os 
import pickle
os.environ["CUDA_VISIBLE_DEVICES"]=""
import warnings
warnings.filterwarnings('ignore')


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [5]:
def MLP(dataset, name,hyperparams,i):
    '''
    Load and return MLP. 
    Attributes: 
        dataset carla.XXX : data to train on 
        name str: dataset name 
    Returns: 
        carla.XXX
    '''
    
    training_params =hyperparams #{"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}
    #if name=='economic':
    #     training_params = {"lr": 0.002, "epochs": 10, "batch_size": 1024, "hidden_size": [18, 9, 3],' num_of_classes':2}
   

    ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
    )

    ml_model.train(
        learning_rate=training_params["lr"],
        epochs=training_params["epochs"],
        batch_size=training_params["batch_size"],
        hidden_size=training_params["hidden_size"],
        force_train=True
        )

    return ml_model

def data(name, not_causal=True, scaler='Identity'):
    '''
    Load and return Toy Dataset.
    Attribute: 
        name str: Name of the Dataset. 

    Returns: 
        (dataset, scm, scm_output): returns Dataset, Structural Causal Model and Structural Causal Model with output Layer.
    '''
    scm = CausalModel(f"{name}")
    scm_output = CausalModel(f"{name}-output")
    if not os.path.isdir(f'./data/{name}'):
        os.mkdir(f'./data/{name}')
    if not os.path.isfile(f'./data/{name}/{name}.csv'):
        print('TRUE 1 ')
        # generate data
        dataset = scm.generate_dataset(10000, False)
        print(f'./data/{name}/{name}.csv')
        dataset.df.to_csv(f'./data/{name}/{name}.csv', index=False)
        #pickle.dump(dataset.train_raw, open(f'./data/{name}/{name}_train_raw.pkl','wb'))
        #pickle.dump(dataset.test_raw, open(f'./data/{name}/{name}_test_raw.pkl','wb'))
        #pickle.dump(dataset.raw, open(f'./data/{name}/{name}_raw.pkl','wb'))
        #pickle.dump(dataset.noise, open(f'./data/{name}/{name}_noise.pkl','wb'))
        if not_causal:
            dataset = pd.read_csv(f'./data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
            continuous_wachter = dataset.drop(columns=['label']).columns
            dataset = CsvCatalog(file_path=f'./data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
    else: 
        if not_causal:
            dataset = pd.read_csv(f'./data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
            continuous_wachter = dataset.drop(columns=['label']).columns
            dataset = CsvCatalog(file_path=f'./data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
        else: 

            dataset = pd.read_csv(f'./data/{name}/{name}.csv')
            continuous_wachter = dataset.drop(columns=['label']).columns
            #TODO Does Scaling Method Idendity make sense ? 
            dataset = CsvCatalog(file_path=f'./data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
    
    return dataset, scm , scm_output


hyperparams={
        #0: {"lr": 0.1, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]},
        #1: {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]},
        0: {"lr": 0.001, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}


    }
    #import 1_Experiment
    #SEED Setting


scaler='MinMax'
    #Load Dataset    
dataset, scm, scm_output=data('economic', True,scaler)

ml_model= MLP(dataset, 'economic',hyperparams[0],0)



Noise Distribution Finished
balance on test set 0.4368, balance on test set 0.4272
Epoch 0/9
----------
train Loss: 0.3632 Acc: 0.8531

test Loss: 0.1929 Acc: 0.9332

Epoch 1/9
----------
train Loss: 0.1340 Acc: 0.9627

test Loss: 0.0916 Acc: 0.9860

Epoch 2/9
----------
train Loss: 0.0788 Acc: 0.9808

test Loss: 0.0610 Acc: 0.9908

Epoch 3/9
----------
train Loss: 0.0599 Acc: 0.9828

test Loss: 0.0523 Acc: 0.9844

Epoch 4/9
----------
train Loss: 0.0497 Acc: 0.9840

test Loss: 0.0402 Acc: 0.9920

Epoch 5/9
----------
train Loss: 0.0436 Acc: 0.9855

test Loss: 0.0425 Acc: 0.9812

Epoch 6/9
----------
train Loss: 0.0402 Acc: 0.9871

test Loss: 0.0309 Acc: 0.9932

Epoch 7/9
----------
train Loss: 0.0362 Acc: 0.9877

test Loss: 0.0328 Acc: 0.9884

Epoch 8/9
----------
train Loss: 0.0361 Acc: 0.9864

test Loss: 0.0466 Acc: 0.9768

Epoch 9/9
----------
train Loss: 0.0326 Acc: 0.9877

test Loss: 0.0621 Acc: 0.9644

